In [1]:
import numpy as np
import pandas as pd
from IPython.display import display


## Ошибочный препроцессинг

In [2]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

<b>Ошибочно</b>

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
model = LinearRegression().fit(X_train_transformed, y_train)
mean_squared_error(y_test, model.predict(X_test))

62.80867119249539

<b>Правильно</b>

In [4]:
X_test_transformed = scaler.transform(X_test)
mean_squared_error(y_test, model.predict(X_test_transformed))

0.902797546636954

In [5]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)


mean_squared_error(y_test, model.predict(X_test))

0.902797546636954

## Утечка данных

In [6]:
import numpy as np
n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

<b>Ошибочно</b>

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Неправильная предварительная обработка: преобразуются все данные
X_selected = SelectKBest(k=25).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, random_state=42)
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train, y_train)


y_pred = gbc.predict(X_test)
accuracy_score(y_test, y_pred)

0.76

<b>Правильно</b>

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
select = SelectKBest(k=25)
X_train_selected = select.fit_transform(X_train, y_train)

gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)

X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.46

In [9]:
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
pipeline = make_pipeline(SelectKBest(k=25),
                         GradientBoostingClassifier(random_state=1))
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.46

In [10]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y)
print(f"Mean accuracy: {scores.mean():.2f}+/-{scores.std():.2f}")

Mean accuracy: 0.45+/-0.07


Советы по предотвращению утечки данных:

1. Всегда изначально разделяйте данные на обучающие и тестовые выборки, особенно перед любыми этапами предварительной обработки.

2. Никогда не включайте тестовые данные в методы <code>fit</code> и <code>fit_transform</code>. Использование всех данных мягко говоря, может привести к чрезмерно оптимистичным оценкам. </br>
И наоборот, метод <code>transform</code> следует использовать как для обучающих, так и для тестовых выборок, поскольку ко всем данным должна применяться одинаковая предварительная обработка. Этого можно достичь, используя <code>fit_transform</code> в обучающей выборке и <code>transform</code> в тестовой.

3. Конвейер scikit-learn - отличный способ предотвратить утечку данных, поскольку он гарантирует, что соответствующий метод выполняется на правильном выборке. Конвейер идеально подходит для использования в функциях перекрестной проверки и настройки гиперпараметров.

## Контроль "случайности"

Алгоритмы используемые в scikit-learn часто, по своей сути, случайны. Например, RandomForestClassifier и  KFold. "Случайность" этих алгоритмов контролируется с помощью их параметра <code>random_state</code>


Для  надежности результатов перекрестной проверки (CV) оставьте <code>random_state</code> равным <code>None</code> или целому числу. 

Передача целых чисел в валидацию CV обычно является самым безопасным и предпочтительным вариантом. 

Передача экземпляров RandomState <code>train_test_split</code> иногда может быть полезна для достижения одинакового результата использования. 

И для CV, и для Split передача целого числа по сравнению с передачей экземпляра (или None) приводит к тонким, но значительным различиям вычислительных продцедур, особенно для процедур CV. Эти различия важно понимать при анализе результатов моделели.

Для воспроизводимости результатов вычислений необходимо исключить любое использование <code>random_state = None</code>.

In [11]:
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
import numpy as np

rng = np.random.RandomState(0)
X, y = make_classification(n_features=5, random_state=rng)
sgd = SGDClassifier(random_state=rng)

print( sgd.fit(X, y).coef_)
print( sgd.fit(X, y).coef_)

[[ 8.85418642  4.79084103 -3.13077794  8.11915045 -0.56479934]]
[[ 6.70814003  5.25291366 -7.55212743  5.18197458  1.37845099]]


In [12]:
X, y = make_classification(n_features=5, random_state=8)
sgd = SGDClassifier(random_state=12)

print( sgd.fit(X, y).coef_)
print( sgd.fit(X, y).coef_)

[[-19.16369217  10.14644031   0.60707996   0.56520545  24.43565214]]
[[-19.16369217  10.14644031   0.60707996   0.56520545  24.43565214]]


In [13]:
from sklearn.model_selection import KFold
import numpy as np

X = y = np.arange(10)
rng = np.random.RandomState(0)
cv = KFold(n_splits=2, shuffle=True, random_state=rng)

for train, test in cv.split(X, y):
    print(train, test)

for train, test in cv.split(X, y):
    print(train, test)

[0 3 5 6 7] [1 2 4 8 9]
[1 2 4 8 9] [0 3 5 6 7]
[0 4 6 7 8] [1 2 3 5 9]
[1 2 3 5 9] [0 4 6 7 8]


И еще о "подводных камнях"

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
import numpy as np

X, y = make_classification(random_state=0)

rf_123 = RandomForestClassifier(random_state=123)
print(cross_val_score(rf_123, X, y))


rf_inst = RandomForestClassifier(random_state=np.random.RandomState(0))
print(cross_val_score(rf_inst, X, y))

[0.85 0.95 0.95 0.9  0.9 ]
[0.9  0.95 0.95 0.9  0.9 ]


Мы видим, что результаты перекрестной проверки rf_123 и rf_inst различаются, как и следовало ожидать, поскольку мы не передали один и тот же параметр random_state. Однако разница между этими оценками более тонкая, чем кажется, и процедуры перекрестной проверки, которые выполнялись с помощью cross_val_score, значительно различаются в каждом случае:

Поскольку rf_123 было передано целое число, каждый вызов функции fit использует один и тот же RNG: это означает, что все случайные характеристики случайного оценщика леса будут одинаковыми для каждого из 5 кратностей процедуры CV. <b>В частности, (случайно выбранное) подмножество функций оценщика будет одинаковым для всех подвыборок.</b>

Поскольку rf_inst был передан экземпляр RandomState, каждый вызов для соответствия начинается с другого RNG. В результате случайный набор функций будет отличаться для каждой подвыбороки.

Хотя наличие постоянного оценщика RNG по подвыборокам не является неправильным по своей сути, мы обычно хотим, чтобы результаты CV были надежными по отношению к "случайности" оценщика. В результате передача экземпляра  RandomState вместо целого числа может быть предпочтительнее, поскольку это позволит оценцику изменяться для каждой подвыборки.

### Клонирование

In [15]:
from sklearn import clone
from sklearn.ensemble import RandomForestClassifier
import numpy as np

X, y = make_classification(random_state=0)

rng = np.random.RandomState(0)
a = RandomForestClassifier(random_state=rng)
b = clone(a)

print(cross_val_score(a, X, y))
print(cross_val_score(b, X, y))

[0.9  0.95 0.95 0.9  0.9 ]
[0.9  0.95 0.95 0.9  0.9 ]


Клоны не являются копиями в прямом сысле слова - это не два различных одинаковых объекта, это один объект с двумя именами, они ссылаются на один и тотже объект. Если бы было передано целое число, a и b были бы точными клонами, и они не влияли бы друг на друга.

## Многократная повторяемость результата

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

rng = np.random.RandomState(18)
X, y = make_classification(random_state=rng)

rf = RandomForestClassifier(random_state=rng)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng)
rf.fit(X_train, y_train).score(X_test, y_test)

0.84

Важно! Не устанавливайте в вашем коде np.random.RandomState(0) дискуссия тут https://stackoverflow.com/questions/5836335/consistently-create-same-random-numpy-array/5837352#comment6712034_5837352

Кстати, а почему всегда одинаково и почему так лучше, чем передать целое число?